In [1]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
from scipy import signal, optimize, stats
from observer import make_kfilter, vibe_fit_freq, get_psd, predict, update, noise_filter

In [2]:
openloop = np.load('keck_tt/OpenLoop_n0088.npy')
centroid_loaded = np.load('keck_tt/Centroid_n0088.npy')
centroid = np.zeros(centroid_loaded.shape)
commands_loaded = np.load('keck_tt/Commands_n0088.npy')
commands = np.zeros(commands_loaded.shape) # to check later if this matches keck_tt/Commands

In [3]:
steps = openloop.shape[0]
f_sampling = 1000
times = np.arange(0, steps/f_sampling, 1/f_sampling)

In [4]:
for i in range(steps):
    if i >= 3:
        commands[i] = 0.6*commands[i-1] + 0.32*commands[i-2] + 0.08*commands[i-3] + 0.1*centroid[i]
    centroid[i+1] = openloop[i] - commands[i]

In [5]:
print("Average deviation in x, open loop: ", sum([np.mean(openloop[:,0]**2)]))

Average deviation in x, open loop:  0.009095664136111736


In [6]:
print("Average deviation in x, standard integrator: ", sum([np.mean(centroid[:,0]**2)]))

Average deviation in x, standard integrator:  0.002200652111463444


In [7]:
# build a Kalman filter based on POLs from the first 1 second
# since it'll be operating on POLs this is actually more accurate than just a physics simulation
# in real life, you'd probably run the standard integrator for 1 second and recover open-loops

# this is going to be garbage, because there's no turbulence fit yet

centroid_kalman = np.zeros(steps)

state, A, P, Q, H, R = make_kfilter(*vibe_fit_freq(noise_filter(get_psd(openloop[:1000][:,0])), N=10))
states_freq = np.zeros((steps, state.size))
for k in range(steps):
    state, P = update(H, P, R, state, openloop[k,0]) # Kalman filter takes in the POL
    states_freq[k] = state
    state, P = predict(A, P, Q, state)
    centroid_kalman[k] = openloop[k,0] - H.dot(states_freq[k])
    
pos_freq = np.array([H.dot(state) for state in states_freq]).flatten()

In [8]:
print("Average deviation in x, Kalman filter: ", sum([np.mean(pos_freq**2)]))

Average deviation in x, Kalman filter:  0.009777012832330498


In [9]:
print(centroid_kalman)

[ 4.92138771e+00  2.91180014e-02 -2.72876701e-02 ...  1.26422814e-04
  1.39513586e-04 -8.32549741e-04]
